In [1]:
# GET PARAMETERS
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern

Training and Hyper-parameter tuning with Ray is not possible


## Gestion des outliers: Estimation du Flux Max d'une station
- De Nombreux outliers présents à Perrache, d'autres assez élevée sur les autres stations. On va essayer de gérer ces outliers. 
    - On prend un Flux maximal enregistré : le Flux in max en 15min enregistré pendant la fête des lumière
        - Le Flux max est enregistré à Bellecour, autour de 2500 personnes / 15 minutes
        - Le second Flux max est enregistré à Charpennes, autour de 1600 personnes / 15 minutes
        - Le 3eme Flux max est enregsitré à Gorge de Loup, autour de 1600 personnes / 15 minutes
        - Aucunes des autres stations n'a dépassée 1400, même pendant la fête des lumières. 

- On choisit donc d'inputer la données, par interpolation linéaire, pour tout les flux dépassant:
    -  2500 pour bellecour
    - 1600 pour charpnnes et gorge de Loup
    - 1400 pour les autres stations.

# Comparaison des flux avec ce qu'on peut trouver sur les validations individuelles:

In [ ]:
import pandas as pd
import glob
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from constants.paths import FOLDER_PATH

list_files = glob.glob(f"{parent_dir}/{FOLDER_PATH}/validations_individuelles/*.txt")

# Load df :


path_txt = list_files[1]
df = pd.read_csv(path_txt, sep = "\t")
df['is_subway'] = df.EMP_CEB.transform(lambda x : x[:3])
df_subway = df[df['is_subway'] == 'VMM']   # Extract Funicular and Subway, starting both by "VMM"

df_subway['subway_mode'] = df_subway.EMP_CEB.transform(lambda x : x[:4])
df_subway = df_subway[df_subway['subway_mode'] != 'VMMF'] # Extract VVMx and remove VMMF (i.e. Subway Lane, excluding Funicular)
df_subway.VAL_DATE = pd.to_datetime(df_subway.VAL_DATE)

if False:
    # Load ref subway
    ref_subway = pd.read_csv(f"{folder_path}/ref_subway.csv").rename(columns = {'MEAN_X' : 'lon','MEAN_Y':'lat'})
    ref_subway = ref_subway[['lon','lat','COD_TRG','LIB_STA_SIFO']]

#Get Subway  
df_subway['subway_mode'] = df_subway.EMP_CEB.transform(lambda x : x[:4])
df_subway = df_subway[df_subway['subway_mode'] != 'VMMF'] # Extract VVMx (i.e. Subway Lane)
df_subway['COD_TRG'] = df_subway.EMP_CEB.transform(lambda x : x[5:8])
df_subway['lane'] = df_subway.EMP_CEB.transform(lambda x : x[3])

In [ ]:
df_subway_agg = df_subway.groupby([pd.Grouper(freq='15min',key='VAL_DATE'),'COD_TRG']).count().iloc[:,0]
df_subway_agg.name = 'Flow'
df_subway_agg = df_subway_agg.reset_index()

#df_subway_agg[~df_subway_agg['COD_TRG'].isin(['CHA','BEL','GOR'])].pivot_table(index = 'VAL_DATE',columns = 'COD_TRG',values='Flow').plot()
df_subway_agg[df_subway_agg['COD_TRG'].isin(['HOT','BEL','CHA','AMP','PER','PAR'])].pivot_table(index = 'VAL_DATE',columns = 'COD_TRG',values='Flow').plot()
#df_subway_agg.pivot_table(index = 'VAL_DATE',columns = 'COD_TRG',values='Flow').plot()

## Génération de la configuration, choix des modification, entrainement et visalisation.
Args: 
--
- station : définie les station qui vont être regardée (visualisation profil time série réelle et prédiction)
- set_spatial_units: défini les stations qui vont être prise en compte lors de l'entrainement. La taille de 'set_spatial_unit' est donc la taille du graphe. Si non précisé, on fait la prédiction sur les 40 STATION du réseau.

In [ ]:
# Init:
dataset_names = ["subway_in"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
station = ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'
# ...

# Modif 
modification = {'epochs' : 200, #100
                'lr':4e-4,
                #'set_spatial_units':  station,       
                           }
# ...

# Training and visu: 
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,
                                                   station=station,modification=modification,
                                                   training_mode_to_visualise=training_mode_to_visualise)

In [4]:
# GET PARAMETERS
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern

# Init:
dataset_names = ["subway_in","subway_out"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
station = ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'
# ...

# Modif 
modification = {'epochs' : 100, #100
                'lr':4e-4,# 4e-4,
                'weight_decay':0.05,
                'dropout':0.1,
                #'set_spatial_units':  station,  
                #   
                #'scheduler':None,
                'scheduler':True,
                'torch_scheduler_milestone': 2,
                'torch_scheduler_gamma':0.99,
                'torch_scheduler_lr_start_factor':0.1,

                'vision_num_heads':4,
                "vision_grn_out_dim":32,
                'vision_model_name': 'VariableSelectionNetwork',
                'vision_concatenation_early':True,   
                'vision_concatenation_late':True,
                           }
# ...

# Training and visu: 
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,
                                                   station=station,modification=modification,
                                                   training_mode_to_visualise=training_mode_to_visualise)

>>>>Model: STGCN; K_fold = 6; Loss function: MSE 



Init Dataset:  torch.Size([7392, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295680 

U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  AMP
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  BEL
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  BRO
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  COR
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  CUI
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  CUS
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  FLA
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Estimated time for training: 5.8min 

Training Throughput:412.99 sequences per seconds
>>> Training complete in: 0:04:55.384199
>>> Training performance time: min 0.05637812614440918 avg 0.07464814186096191 seconds (+/- 0.011679907646017843)
>>> Loading performance time: min 0.0005350112915039062 avg 0.054818783674027705 seconds (+/- 0.1422254919270231)
>>> Forward performance time: 0.03309917523338961 seconds (+/- 0.010381782725073323)
>>> Backward performance time: 0.036790994920837004 seconds (+/- 0.0048331951742535)
>>> Plotting performance time: 4.758738508128156e-06 seconds (+/- 7.979797503624394e-06)
>>> Saving performance time: 1.0893031723645268 seconds (+/- 0.1678707398317103)
>>> PI-tracking performance time: 7.145332567619555e-06 seconds (+/- 1.4053749012472465e-05)
>>> Scheduler-update performance time: 7.530655523743292e-05 seconds (+/- 0.000139997121338504)
>>> Peak Power during training: 151.808 W)
>>> Validation time: 0:00:00.353289
Proportion of time consumed for Load

Loading BokehJS ...

## Pattern d'erreur selon les stations : 

In [ ]:
# Init
for training_mode in training_mode_to_visualise:
    min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
    limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
    fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                freq='1h',
                                                min_flow=min_flow,
                                                figsize = (30,5*len(ds.spatial_unit)),
                                                limit_percentage_error = limit_percentage_error)

In [1]:
# GET PARAMETERS
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern

dataset_names = ["subway_in","subway_out"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = ['BEL','PAR','AMP','FLA']   # 'BON'  #'GER'
# ...

modification = {'epochs' : 200, #100
                'lr':4e-4,
                #'set_spatial_units': station,#['CHA','GER','BON','SOI'],
                #'TE_concatenation_late':False,
                #'TE_concatenation_early':True,   
                #'TE_embedding_dim' : 16, # 3
                #'TE_multi_embedding' : True , 
                'vision_model_name': 'VariableSelectionNetwork',
                'vision_concatenation_early':True,   
                'vision_concatenation_late':True,
                           }
training_mode_to_visualise = ['test']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,
                                                   station = station,modification=modification,training_mode_to_visualise=training_mode_to_visualise)

# Init
if False:
    for training_mode in training_mode_to_visualise:
        min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
        limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
        fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                    freq='1h',
                                                    min_flow=min_flow,
                                                    figsize = (30,5*len(ds.spatial_unit)),
                                                    limit_percentage_error = limit_percentage_error)

Training and Hyper-parameter tuning with Ray is not possible
>>>>Model: STGCN; K_fold = 6; Loss function: MSE 

Init Dataset:  torch.Size([7392, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295680 

U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  AMP
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  BEL
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  BRO
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  COR
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  CUI
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  CUS
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/

Loading BokehJS ...

In [ ]:
trainer.args.epochs = 20
trainer.train_and_valid(mod = 1000,mod_plot = None) 
print(trainer.performance['valid_metrics']['mse'],trainer.performance['valid_metrics']['mae'],trainer.performance['valid_metrics']['mape'])
print(trainer.performance['test_metrics']['mse'],trainer.performance['test_metrics']['mae'],trainer.performance['test_metrics']['mape'])

import pandas as pd 
pd.DataFrame(dict(valid = trainer.valid_loss)).iloc[-120:].plot()

In [ ]:
# GET PARAMETERS
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern

dataset_names = ["subway_in","subway_out"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = ['BEL','PAR','AMP','FLA']   # 'BON'  #'GER'
# ...

modification = {'epochs' : 100, #100
                'lr':4e-4,
                #'set_spatial_units': station,#['CHA','GER','BON','SOI'],
                #'TE_concatenation_late':False,
                #'TE_concatenation_early':True,   
                #'TE_embedding_dim' : 16, # 3
                #'TE_multi_embedding' : True , 
                'vision_model_name': 'VariableSelectionNetwork',
                'vision_concatenation_early':False,   
                'vision_concatenation_late':True,
                           }
training_mode_to_visualise = ['test']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,
                                                   station = station,modification=modification,training_mode_to_visualise=training_mode_to_visualise)

# Init
if False:
    for training_mode in training_mode_to_visualise:
        min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
        limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
        fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                    freq='1h',
                                                    min_flow=min_flow,
                                                    figsize = (30,5*len(ds.spatial_unit)),
                                                    limit_percentage_error = limit_percentage_error)

In [ ]:
print(trainer.performance['valid_metrics']['mse'],trainer.performance['valid_metrics']['mae'],trainer.performance['valid_metrics']['mape'])
print(trainer.performance['test_metrics']['mse'],trainer.performance['test_metrics']['mae'],trainer.performance['test_metrics']['mape'])

In [ ]:
trainer.train_and_valid(mod = 1000,mod_plot = None) 
print(trainer.performance['valid_metrics']['mse'],trainer.performance['valid_metrics']['mae'],trainer.performance['valid_metrics']['mape'])
print(trainer.performance['test_metrics']['mse'],trainer.performance['test_metrics']['mae'],trainer.performance['test_metrics']['mape'])

In [ ]:
from plotting.plotting import plot_attn_weight
from constants.paths import CALENDAR_TYPE
nb_calendar_data = len(CALENDAR_TYPE)

training_mode = 'test'
temporal_agg = 'weekday_hour' # 'hour' # 'weekday' # 'weekday_hour'
save = 'attn_weight'
plot_attn_weight(trainer,nb_calendar_data,ds,training_mode,temporal_agg,save)


In [155]:
import torch.nn as nn
import math
import torch 

class FastGLU(nn.Module):
    def __init__(self,input_size):
        super(FastGLU,self).__init__()
        self.input_size = input_size
        self.dense = nn.Linear(input_size,input_size*2)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        x = self.dense(x)
        out = x[...,:self.input_size] * self.sigmoid(x[...,self.input_size:])
        return(out)


class GRN(nn.Module):
    def __init__(self,input_size,grn_h_dim,grn_out_dim,contextual_static_dim,dropout):
        super(GRN,self).__init__()
        self.input_size = input_size
        self.grn_out_dim = grn_out_dim
        if self.input_size != self.grn_out_dim:
            self.align = nn.Linear(self.input_size,self.grn_out_dim)
        self.dense1 = nn.Linear(self.input_size,grn_h_dim) 
        if contextual_static_dim is not None:
            self.dense_contextual = nn.Linear(contextual_static_dim,grn_h_dim)
        self.dense2 = nn.Linear(grn_h_dim,self.grn_out_dim)
        self.elu = nn.ELU()
        self.dropout = nn.Dropout(dropout)
        self.gate = FastGLU(grn_out_dim)
        # LayerNorm
        self.layer_norm = nn.LayerNorm(grn_out_dim)
    def forward(self,x,x_c=None):
        '''
        Inputs: 
        -------
        x : 3-th order tensor:  [B,C,input_size]     (input_size = L)
        x_c : 2-th order tensor:  [B,z_contextual] 

        Ouputs: 
        -------
        output : 3-th order tensor:  [B,C,grn_out_dim] 
        '''
        # Align init x_trafic with projection of this model  
        x_init = x
        if self.input_size != self.grn_out_dim:
            x_init = self.align(x_init)

        # Embedding of x_trafic
        x = self.dense1(x)

        # Add known contextual if needed : 
        if x_c is not None:
            x_c = self.dense_contextual(x_c)  
            x  = x+x_c

        x = self.gate(self.dropout(self.dense2(self.elu(x))))
        
        
        # Add & LayerNorm
        x = x+x_init  # Add
        x = self.layer_norm(x)  # LayerNorm
        return(x)
    

class AttentionGRU(nn.Module):
    def __init__(self, input_length1,input_length2, d_model,grn_h_dim,dropout):
        super(AttentionGRU, self).__init__()

        print('\n>>>>>>>>> input_length1,grn_h_dim,d_model,input_length2 :',input_length1,grn_h_dim,d_model,input_length2)
        self.gru  = GRN(input_length1,grn_h_dim,d_model,input_length2,dropout)
        self.attention = ScaledDotProduct_i(input_length1, d_model)

    def forward(self, x_trafic,x_dynamic,x_known):
        ''''
        x_trafic : 2-th order Tensor : historical sequence of trafic flow [B,L]
        x_context : 3-th order Tensor : R Embedding or P_i contextual data associated to the spatial unit of x_trafic [B,P_i,L]
        '''
        query = self.gru(x_trafic,x_known)
        enhanced_x,attn_weights = self.attention(query,x_dynamic,x_dynamic)
        return enhanced_x,attn_weights
    

class ScaledDotProduct_i(nn.Module):
    def __init__(self, input_length1, d_model):
        super(ScaledDotProduct_i, self).__init__()
        self.proj1 = nn.Linear(input_length1, d_model)
        self.d_model = d_model

        self.W_q = nn.Parameter(torch.cuda.FloatTensor(d_model, d_model)) if torch.cuda.is_available() else nn.Parameter(torch.FloatTensor(d_model, d_model))
        self.W_k = nn.Parameter(torch.cuda.FloatTensor(d_model, d_model)) if torch.cuda.is_available() else nn.Parameter(torch.FloatTensor(d_model, d_model))
        self.W_v = nn.Parameter(torch.cuda.FloatTensor(d_model, d_model)) if torch.cuda.is_available() else nn.Parameter(torch.FloatTensor(d_model, d_model))

        self.softmax = nn.Softmax(dim = -1)

    def forward(self, query,key,values):
        '''
        query: From x_traffic
        key : From x_dynamic
        values: From x_dynamic
        '''

        key = self.proj1(key)

        # Q,K,V
        if query.dim() == 2:
            query = query.unsqueeze(1)
        Q = torch.matmul(query,self.W_q)
        K = torch.matmul(key,self.W_k)
        V = torch.matmul(key,self.W_v)
        K = K.permute(0,2,1)

        scaled_compat = torch.matmul(Q,K)*1.0/math.sqrt(self.d_model)
        attn_weights = self.softmax(scaled_compat)
        context = torch.matmul(attn_weights,V)
        context = torch.sum(context, dim=1)
        
        return context,attn_weights

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Supposons que attn_weights est votre tenseur de poids d'attention
# attn_weights : Shape [B, 1, P]
# Prenons le premier exemple du batch
attn_weights_example = attn_weights[0].squeeze(0)  # Shape [P]

# Normaliser les poids d'attention pour la visualisation
attn_weights_example = attn_weights_example.detach().cpu().numpy()  # Convertir en numpy

# Créer une figure
plt.figure(figsize=(10, 6))

# Afficher les poids d'attention
plt.bar(range(len(attn_weights_example)), attn_weights_example, color='skyblue')
plt.title('Poids d\'attention pour l\'exemple 1')
plt.xlabel('Séries temporelles')
plt.ylabel('Poids d\'attention')

# Ajouter des ticks pour les séries temporelles
plt.xticks(range(len(attn_weights_example)), [f'Série {i}' for i in range(len(attn_weights_example))], rotation=45)

# Afficher la figure
plt.tight_layout()
plt.show()

In [ ]:
from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern

dataset_names = ["subway_in"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = ['PAR']   # 'BON'  #'GER'

modification = {'epochs' : 250, #100
                'lr':1e-4,
                #'set_spatial_unit': station
                }

training_mode_to_visualise = ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,
                                                  station = station,
                                                  modification=modification,
                                                  training_mode_to_visualise=training_mode_to_visualise)

In [ ]:
# Init
for training_mode in training_mode_to_visualise:
    min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
    limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
    fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                freq='1h',
                                                min_flow=min_flow,
                                                figsize = (30,5*len(ds.spatial_unit)),
                                                limit_percentage_error = limit_percentage_error)

In [ ]:
import torch
import pandas as pd
index_perrache = list(ds.spatial_unit).index('PER')
inputs = [[x,y,x_c] for  x,y,x_c in ds.dataloader['test']]
X = torch.cat([x for x,_,_ in inputs],0)
Y = torch.cat([y for _,y,_ in inputs],0)
#X_c = torch.cat([x_c for x,_,_ in inputs],-1)
l = 96
pd.DataFrame({'PER_TRUE': Y[:,index_perrache,0].detach().clone().reshape(-1),
              #'t-1': X[:,index_perrache,-1].detach().clone().reshape(-1),
              #'t-2': X[:,index_perrache,-2].detach().clone().reshape(-1),
              #'t-3': X[:,index_perrache,-3].detach().clone().reshape(-1),
              #'t-4': X[:,index_perrache,-4].detach().clone().reshape(-1),
              #'t-5': X[:,index_perrache,-5].detach().clone().reshape(-1),
              #'t-6': X[:,index_perrache,-6].detach().clone().reshape(-1),
              't-d': X[:,index_perrache,-7].detach().clone().reshape(-1),
              }).iloc[l:l+7*l].plot()


In [ ]:
from examples.train_and_visu_non_recurrent import evaluate_config
dataset_names = ["subway_in","calendar"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = 'GER'  # 'BON'  #'GER'
apps = ['Google_Maps']# ['Instagram','Twitter','Google_Maps'] # 'Instagram'  # 'Twitter' # 'Google_Maps' # 'Facebook'


transfer_modes = ['DL'] # ['DL'] # ['UL'] # ['DL','UL']
type_POIs = ['stadium'] #['stadium','station'] #['stadium','nightclub'] #['stadium']
spatial_units = ['Matmut Stadium Gerland','GER']  #spatial_units = ['Lou_rugby']  # ['Astroballe'] #['Lou_rugby','Ninkasi_Kao'] #['Lou_rugby'] #['Ninkasi_Kao'] 
expanded = '_expanded' # '' # '_expanded' # ''
POI_or_stations = ['POI']# ['POI','station'] # 'station'
modification = {'epochs' : 400, #100
                'lr':1e-4,
                'set_spatial_units': ['CHA','GER','BON','SOI'],#['CHA','GER','BON','SOI'],
                'TE_concatenation_late':False,
                'TE_concatenation_early':True,   
                'TE_embedding_dim' : 2,
                'TE_multi_embedding' : True ,         
                           }
training_mode_to_visualise = ['test']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,transfer_modes= transfer_modes,
                                                   type_POIs = type_POIs,spatial_units = spatial_units,apps = apps,POI_or_stations = POI_or_stations,
                                                   expanded=expanded,modification=modification,training_mode_to_visualise=training_mode_to_visualise)

## Evaluate on non recurrent event: 
### Visualisation: 
#### Cas `subway_in` stade du Lou Gerland avec un match de Rugby : 

## HP tuning: 

In [ ]:
from examples.Total_evaluation_of_model import HP_and_valid_one_config,hyperparameter_tuning
from examples.benchmark import local_get_args
import pickle

model_name = 'STGCN' #'CNN'
dataset_for_coverage = ['subway_in','netmob_POIs'] 
dataset_names = ['calendar']
vision_model_name = None

args,_,_ = local_get_args(model_name,
                        args_init = None,
                        dataset_names=dataset_names,
                        dataset_for_coverage=dataset_for_coverage,
                        modification = {'ray':True,
                                        'grace_period':2,
                                        'HP_max_epochs':10,
                                        'evaluate_complete_ds' : True,
                                        'set_spatial_units' : ['BON','SOI','GER','CHA'],
                                        'vision_model_name': None
                                        })

# Init 
epochs_validation = 30
num_samples = 10

# HP and evaluate K-fold best config
#HP_and_valid_one_config(args,epochs_validation,num_samples)
analysis,trial_id = hyperparameter_tuning(args,num_samples)
print('trial_id: ',trial_id)



## Load HP tuning, have a look on the best configurations:

In [ ]:
pd.read_csv(path_csv)['']

In [ ]:
import pickle 
import pandas as pd 

# 2000 Trial et 100 Epochs: 
trial_ids = ['subway_in_calendar_STGCN_MSELoss_2024_12_19_07_15_69445','subway_in_STGCN_MSELoss_2024_12_19_21_10_79120']

# 'subway_in_STGCN_MSELoss_2024_12_11_00_16_42861'   -> 'Seulement' 400 trials, et 50 epochs 
for trial_id in trial_ids:
    columns_to_drop = ['time_this_iter_s', 'done', 'date', 
                    'timestamp', 'time_total_s', 'pid', 'hostname', 'node_ip',
                    'time_since_restore', 'iterations_since_restore', 'experiment_tag']
    path_csv = f'save/HyperparameterTuning/{trial_id}.csv'
    path_pickle = 'save/HyperparameterTuning/model_args.pkl'

    df_hp_tuning = pd.read_csv(path_csv)
    model_args = pickle.load(open(path_pickle,'rb'))


    metric = 'Loss_model'

    dict_rename = {key: key.split('/')[-1] for key in df_hp_tuning.columns}
    df_hp_tuning = df_hp_tuning.rename(columns = dict_rename)
    df_hp_tuning = df_hp_tuning.drop(columns = columns_to_drop)


    k=10
    print(df_hp_tuning.shape)
    display(df_hp_tuning.sort_values(metric).iloc[:k])

In [ ]:
save_folder = 'save/K_fold_validation/training_with_HP_tuning'

for trial_id in trial_ids:
    print(f'\n{trial_id}')
    metric_csv = f'{save_folder}/METRICS_{trial_id}.csv'
    df_metrics_k_fold_valid = pd.read_csv(metric_csv,index_col =0)

    losses_csv = f'{save_folder}/Losses_{trial_id}.csv'
    df_losses_k_fold_valid = pd.read_csv(losses_csv,index_col =0)

    display(df_metrics_k_fold_valid)
    usefull_columns = [c for c in df_losses_k_fold_valid.columns if 'valid' in c]
    df_losses_k_fold_valid[usefull_columns].iloc[10:].plot()

In [ ]:
import numpy as np 
from examples.train_model_on_k_fold_validation import train_valid_K_models,get_model_metrics
from examples.benchmark import local_get_args


# TE modification: 
TE_modification = {'TE_concatenation_early':True,
                  'TE_multi_embedding':True,
                  'TE_concatenation_late': False,
                  'TE_out_h_dim':4,
                  'TE_embedding_dim':4
               }


for dataset_names,te_modif in zip([['subway_in','calendar'],['subway_in']],[TE_modification,{}]):
   ### === Modification :
   modification = {'evaluate_complete_ds':True,
                  'temporal_h_dim':32,
                  'spatial_h_dim':8,
                  'output_h_dim':64,
                  'dropout':5e-2,
                  'weight_decay':5e-3,
                  'lr':5e-4,
                  'epochs':2
                  }
   modification.update(te_modif)
   ### _____________________

   # Load Config
   args,_,_ = local_get_args(model_name= 'STGCN',
                           args_init=None,
                           dataset_names= dataset_names,
                           dataset_for_coverage= ['subway_in','netmob_POIs'] ,
                           modification=modification
                           )

   # Train, Valid: K-fold and complete ds 
   trainer,args,valid_losses,training_mode_list,metric_list,df_loss = train_valid_K_models(args,
                                                                                          folds=np.arange(args.K_fold),
                                                                                          hp_tuning_on_first_fold=True,
                                                                                          trial_id= None,
                                                                                          save_folder=None)
   # Get metrics: 
   df_results,df_metrics =  get_model_metrics(trainer,args,valid_losses,training_mode_list,metric_list)

   display(df_results)
   display(df_metrics)

   df_losses = pd.DataFrame({'Valid_loss':trainer.valid_loss,
               'Train_loss':trainer.train_loss
               })[10:]

   df_losses.plot()

In [ ]:
from examples.train_model_on_k_fold_validation import load_configuration
from examples.train_and_visu_non_recurrent import evaluate_config

trial_id = 'calendar_STGCN_MSELoss_2024_12_17_21_18_13719'
best_args,folds = load_configuration(trial_id,load_config=True,epochs=None)

dict_keys_embedding = {'TE_concatenation_order/concatenation_late':'TE_concatenation_late',
            'TE_concatenation_order/concatenation_early':'TE_concatenation_early',
             'TE_fc1/fc1':'TE_fc1', 
             'TE_fc1/fc2':'TE_fc2', 
             'TE_fc1/activation_fc1':'TE_activation_fc1',
             }

for key in dict_keys_embedding.keys():
    value = vars(best_args)[key]
    new_key = dict_keys_embedding[key]
    setattr(best_args,new_key,value)


modification = vars(best_args)


dataset_names = ["calendar"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = 'GER'  # 'BON'  #'GER'
apps = ['Google_Maps']# ['Instagram','Twitter','Google_Maps'] # 'Instagram'  # 'Twitter' # 'Google_Maps' # 'Facebook'


transfer_modes = ['DL'] # ['DL'] # ['UL'] # ['DL','UL']
type_POIs = ['stadium'] #['stadium','station'] #['stadium','nightclub'] #['stadium']
spatial_units = ['Matmut Stadium Gerland','GER']  #spatial_units = ['Lou_rugby']  # ['Astroballe'] #['Lou_rugby','Ninkasi_Kao'] #['Lou_rugby'] #['Ninkasi_Kao'] 
expanded = '_expanded' # '' # '_expanded' # ''
POI_or_stations = ['POI']# ['POI','station'] # 'station'

(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,transfer_modes= transfer_modes,
                                                   type_POIs = type_POIs,spatial_units = spatial_units,apps = apps,POI_or_stations = POI_or_stations,
                                                   expanded=expanded,modification=modification,station=station)

In [ ]:
dataset_names = ["calendar"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
vision_model_name =  None

station = 'GER'  # 'BON'  #'GER'
apps = ['Google_Maps']# ['Instagram','Twitter','Google_Maps'] # 'Instagram'  # 'Twitter' # 'Google_Maps' # 'Facebook'


transfer_modes = ['DL'] # ['DL'] # ['UL'] # ['DL','UL']
type_POIs = ['stadium'] #['stadium','station'] #['stadium','nightclub'] #['stadium']
spatial_units = ['Matmut Stadium Gerland','GER']  #spatial_units = ['Lou_rugby']  # ['Astroballe'] #['Lou_rugby','Ninkasi_Kao'] #['Lou_rugby'] #['Ninkasi_Kao'] 
expanded = '_expanded' # '' # '_expanded' # ''
POI_or_stations = ['POI']# ['POI','station'] # 'station'
modification = {'epochs' : 20, #100
                'lr':1e-4,
                'temporal_h_dim' : 32,
                'spatial_h_dim' : 16,
                'output_h_dim' : 32,
                'TE_embedding_dim': 16,
                'set_spatial_units': ['CHA','GER','BON','SOI'],
                'TE_out_h_dim': 16,
                'TE_concatenation_late':False,
                'TE_concatenation_early':True,
                           }
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,vision_model_name,transfer_modes= transfer_modes,
                                                   type_POIs = type_POIs,spatial_units = spatial_units,apps = apps,POI_or_stations = POI_or_stations,
                                                   expanded=expanded,modification=modification)


# lr 5e-3
# 32 / 32 / 32 - 3.5e-2   , 380 000
# 32 / 16 / 32 - 3.5e-2   , 380 000

#### Cas `subway_out` stade du Lou Gerland avec un match de Rugby : 

#### Cas `subway_in` Laurent Bonnevay Astroballe avec un match de Basket : 

#### Cas `subway_out` Laurent Bonnevay Astroballe avec un match de Basket : 

## Maintenant on va intégrer les données NetMob et voir si elles permettent de réduire l'erreur de prédiction :

In [1]:
import os 
import pickle 
import pandas as pd 

trial_id = 'subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2024_12_07_02_55_5679'
#'subway_in_calendar_STGCN_MSELoss_2024_12_12_15_51_46099'
#'subway_in_calendar_STGCN_MSELoss_2024_12_12_14_16_71587'

path_csv = f'save/HyperparameterTuning/{trial_id}.csv'
path_pickle = 'save/HyperparameterTuning/model_args.pkl'
df_hp_tuning = pd.read_csv(path_csv)
model_args = pickle.load(open(path_pickle,'rb'))

metric = '_metric/Loss_model'

best_model = df_hp_tuning.sort_values(metric).iloc[0]
HP_args = [indx.replace('config/', '') for indx in best_model.index if 'config/' in indx]
args = model_args['model'][trial_id]['args']

In [ ]:
print(df_hp_tuning.columns)

df_hp_tuning.sort_values(metric)[['_metric/Loss_model', 'config/lr', 'config/weight_decay',
       'config/dropout', 'config/vision_vision_grn_out_dim']].iloc[:30]

In [ ]:
dataset_names = ["subway_in","netmob_image_per_station"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
vision_model_name =  'ImageAvgPooling'

(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,vision_model_name)

#### Prediction avec uniquement NetMob, sans subway-in: 

#### Prediction avec uniquement Calendar, sans subway-in: 